In [1]:
import tensorflow as tf
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models, optimizers


In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [ ]:
# Set file paths
fire_dir = r'C:\Users\90544\Downloads\archive (1)\Data\Train_Data\Fire'
non_fire_dir = r'C:\Users\90544\Downloads\archive (1)\Data\Train_Data\Non_Fire'

In [3]:
# Upload and labeling images
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = load_img(img_path, target_size=(128, 128))
        img_array = img_to_array(img) / 255.0  # Normalizasyon
        images.append(img_array)
        labels.append(label)
    return np.array(images), np.array(labels)


In [ ]:
fire_images, fire_labels = load_images_from_folder(fire_dir, 1)  # 1: Fire class
non_fire_images, non_fire_labels = load_images_from_folder(non_fire_dir, 0)  # 0: Non-fire class


In [ ]:
# concatenate images and tags
X = np.concatenate([fire_images, non_fire_images], axis=0)
y = np.concatenate([fire_labels, non_fire_labels], axis=0)

In [ ]:
# divide the data into 70% train, 15% validation, 15% test.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.176, random_state=42)  # %15/(1-%15)=%17.6


In [4]:
print(f'Train set: {len(X_train)}')
print(f'Validation set: {len(X_val)}')
print(f'Test set: {len(X_test)}')


Train set: 4018
Validation set: 859
Test set: 861


In [ ]:
# Create model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
# Customizing the Optimizer (with learning rate)
learning_rate = 0.001  # Değiştirmek istediğin değeri buraya yaz
optimizer = optimizers.Adam(learning_rate=learning_rate)

In [ ]:
# Compile model
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [5]:
# Training model with setting batch size
batch_size = 32  
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=batch_size,
    validation_data=(X_val, y_val)
)

# Evaluation of training and validation accuracy
val_loss, val_acc = model.evaluate(X_val, y_val)
test_loss, test_acc = model.evaluate(X_test, y_test)

print(f'Validation Accuracy: {val_acc}, Test Accuracy: {test_acc}')


Epoch 1/20
126/126 [==============================] - 34s 43ms/step - loss: 0.2695 - accuracy: 0.8890 - val_loss: 0.1643 - val_accuracy: 0.9371
Epoch 2/20
126/126 [==============================] - 5s 38ms/step - loss: 0.1840 - accuracy: 0.9306 - val_loss: 0.1583 - val_accuracy: 0.9418
Epoch 3/20
126/126 [==============================] - 5s 38ms/step - loss: 0.1463 - accuracy: 0.9445 - val_loss: 0.1596 - val_accuracy: 0.9348
Epoch 4/20
126/126 [==============================] - 5s 38ms/step - loss: 0.1344 - accuracy: 0.9515 - val_loss: 0.1265 - val_accuracy: 0.9523
Epoch 5/20
126/126 [==============================] - 5s 38ms/step - loss: 0.1272 - accuracy: 0.9545 - val_loss: 0.1650 - val_accuracy: 0.9534
Epoch 6/20
126/126 [==============================] - 7s 54ms/step - loss: 0.0949 - accuracy: 0.9654 - val_loss: 0.0939 - val_accuracy: 0.9569
Epoch 7/20
126/126 [==============================] - 5s 41ms/step - loss: 0.0866 - accuracy: 0.9686 - val_loss: 0.1385 - val_accuracy: 0.954

In [8]:
model.save("fire_weight.h5")